In [2]:
from config.settings import SRC_DIR, DATA_DIR
import os

# Change working directory
os.chdir(SRC_DIR)

# Define file path
file_dir = DATA_DIR / 'invoices' / 'valid_invoices'
files = os.listdir(file_dir)
files

['invoice.pdf',
 'ISS.pdf',
 'NEWFO-INV-7478.pdf',
 'PerfectMatch.pdf',
 'PO139090 GP.pdf',
 'SKonica Sit23112709230.pdf',
 'skonica3-4.pdf']

In [3]:
import base64

class DataIngestion:
    def __init__(self):
        pass

    @staticmethod
    def transform(file_path):
        '''Reads a PDF file and returns its base64 encoded content.'''
        with open(file_path, "rb") as pdf_file:
            pdf_data = pdf_file.read()
        return base64.standard_b64encode(pdf_data).decode("utf-8")

In [4]:
import re
import json

class Invoice:
    def __init__(self, data):
        self.data = data
    
    def __getitem__(self, key):
        '''Returns the value of the key from the data dictionary.'''
        return self.data.get(key, None)
    
class InvoiceExtractor:
    def __init__(self):
        pass

    def extract(self, response_text):
        '''Extracts and merges JSON data from model response using balanced brace parsing.'''
        def extract_json_objects(text):
            objs = []
            brace_count = 0
            start = None
            for i, ch in enumerate(text):
                if ch == '{':
                    if brace_count == 0:
                        start = i
                    brace_count += 1
                elif ch == '}':
                    brace_count -= 1
                    if brace_count == 0 and start is not None:
                        objs.append(text[start:i+1])
            return objs

        json_blocks = extract_json_objects(response_text)
        if not json_blocks:
            raise ValueError("No valid JSON found in response")
        combined_data = {}
        for block in json_blocks:
            data = json.loads(block)
            combined_data.update(data)
        return Invoice(combined_data)

In [5]:
import google.generativeai as genai
import os
from IPython.display import display, HTML
from json2html import json2html
from config.invoice_config import get_prompt
from dotenv import load_dotenv
load_dotenv()
GCP_KEY = os.getenv("GCP_KEY")

class OCR_Model:
    def __init__(self, model="gemini-2.0-flash-exp"):
        genai.configure(api_key=GCP_KEY)
        self.model = genai.GenerativeModel(model)

    def _predict(self, data):
        '''Generates response using the model.'''
        response = self.model.generate_content(
            [{"mime_type": "application/pdf", "data": data}, get_prompt()]
        )
        return response.text

    def extract(self, data):
        '''Extracts invoice data from the response.'''
        response_text = self._predict(data)
        extractor = InvoiceExtractor()
        return extractor.extract(response_text)

    def display(self, invoice, html=False):
        '''Displays the extracted invoice data.'''
        if html:
            html_table = json2html.convert(json=invoice.data)
            display(HTML(html_table))
        else:
            display(invoice.data)

In [6]:
from config.settings import ROOT_DIR
import os
import json
from tqdm.auto import tqdm

# Define output folder path
output_folder = ROOT_DIR / 'invoice_outputs' / 'gemini-2.0-flash-exp'
output_folder.mkdir(parents=True, exist_ok=True) 

def save_file(invoice, file_name):
    json_file_path = output_folder / f"{file_name}.json"

    # Ensure the full path exists
    json_file_path.parent.mkdir(parents=True, exist_ok=True)

    try:
        with open(str(json_file_path), "w", encoding="utf-8") as json_file:
            json.dump(invoice, json_file, indent=4)
        print(f"JSON saved at: {json_file_path}")
    except PermissionError as e:
        print(f"Permission error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

In [6]:
# Loop through files and process them
for file in tqdm(files):
    file_path = file_dir / file

    try:
        data_ingestion = DataIngestion()
        data = data_ingestion.transform(file_path)

        ocr_model = OCR_Model()
        invoice = ocr_model.extract(data)

        file_name = file_path.stem
        save_file(invoice.data, file_name)
    except Exception as e:
        print(f"Error processing {file}: {e}")

  0%|          | 0/7 [00:00<?, ?it/s]

Permission error: [Errno 13] Permission denied: 'C:\\Users\\ai.users\\imscanCS\\invoice_outputs\\gemini-2.0-flash-exp\\invoice.json'
JSON saved at: C:\Users\ai.users\imscanCS\invoice_outputs\gemini-2.0-flash-exp\ISS.json
JSON saved at: C:\Users\ai.users\imscanCS\invoice_outputs\gemini-2.0-flash-exp\NEWFO-INV-7478.json
JSON saved at: C:\Users\ai.users\imscanCS\invoice_outputs\gemini-2.0-flash-exp\PerfectMatch.json
JSON saved at: C:\Users\ai.users\imscanCS\invoice_outputs\gemini-2.0-flash-exp\PO139090 GP.json
JSON saved at: C:\Users\ai.users\imscanCS\invoice_outputs\gemini-2.0-flash-exp\SKonica Sit23112709230.json
Error processing skonica3-4.pdf: Invalid control character at: line 173 column 6 (char 5888)


In [7]:
import json
import pandas as pd
from pathlib import Path

json_folder = Path(ROOT_DIR) / "invoice_outputs" / "gemini-2.0-flash-exp"
excel_output_path = json_folder / "structured_invoices.xlsx"

# Initialize an Excel writer
with pd.ExcelWriter(excel_output_path, engine="openpyxl") as writer:
    for json_file in json_folder.glob("*.json"):
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        # Convert JSON to a readable format (list of dictionaries)
        df = pd.DataFrame([{"JSON Data": json.dumps(data, indent=2)}])
        
        # Use the file name as the sheet name
        sheet_name = json_file.stem[:30]  # Excel sheet name max length = 31
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Excel file saved at: {excel_output_path}")

Excel file saved at: C:\Users\ai.users\imscanCS\invoice_outputs\gemini-2.0-flash-exp\structured_invoices.xlsx
